In [ ]:
!pip install sentence_transformers
!pip install language-tool-python
import pandas as pd
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Check similatiries between two strings
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-mpnet-base-v2')

def similar(a, b):
  return re.findall("\d+\.\d+", str(util.dot_score(a, b)))[0]

In [ ]:
#RESET THE DATAGRAME; BE CAREFULL!!!

data = {"Joke" : ["Why was the math book sad? Because it had so many problems"], "Chiste": ["Van dos y se cae el del medio"]}
post = pd.DataFrame(data)
post.to_excel("/content/drive/MyDrive/Trabajo Fin de Grado/Final-Degree-Proyect/Wrappers/Jokes.xlsx")

In [ ]:
#Read the data
#post = pd.read_excel("/content/drive/MyDrive/Trabajo Fin de Grado/TFG/Wrappers/Jokes.xlsx", index_col=0)

In [ ]:
#English Jokes

#Beutifull Soup
flag = 0
page = 0

#Extract 20 jokes
while flag < 20:

  #specify URL
  if page == 0:
    url = "http://www.laughfactory.com/jokes/clean-jokes/"
  else:
    url = "http://www.laughfactory.com/jokes/clean-jokes/{}".format(page)

  #get the HTML from the page and assign it to the html variable
  html = urlopen(url)

  #parse the HTML and assign it to the variable 'soup'
  soup = bs(html.read(), 'html.parser')

  #get the text of the joke and assign to the variable "chiste"
  joke = soup.find("div", class_="jokes").text
  for jok in soup.find_all("div", class_ = "jokes"):
    #Avanzamos de pagina para la siguiente url
    page += 1
    joke = jok.find("div", class_ = "joke-text").text
    #que sean chister cortos
    if len(joke) < 300:
      escape = False
      for chist in post.Chiste:
          if similar(chist, joke) > 0.9:
              escape = True
      if escape == False:
        if flag == 0:
          dat = {"Jokes": [joke.strip()]}
          jokes = pd.DataFrame(dat)
        else:   
          dat = {"Jokes": joke.strip()}
          jokes = jokes.append(dat, ignore_index=True)
        flag += 1

In [ ]:
#Chiste español 
#Beutifull Soup
flag = 0
#specify URL
url = "http://www.chistes.com/ChisteAlAzar.asp?n=4"

#Sacar 100 chistes
while flag < 20:
  #get the HTML from the page and assign it to the html variable
  html = urlopen(url)

  #parse the HTML and assign it to the variable 'soup'
  soup = bs(html.read(), 'html.parser')

  #get the text of the joke and assign to the variable "chiste"
  joke = soup.find("div", class_="chiste").text
  
  #que sean chister cortos
  if len(joke) < 300:
    escape = False
    for chist in post.Chiste:
        if similar(chist, joke) > 0.9:
            escape = True
    if escape == False:
      if flag == 0:
        dat = {"Chiste": [joke]}
        chistes = pd.DataFrame(dat)
        flag += 1
      else:
        escape2 = False
        for chi in chistes.Chiste:
          if similar(joke,chi) > 0.9:
            escape2 = True
        if escape2 == False:
          dat = {"Chiste": joke}
          chistes = chistes.append(dat, ignore_index=True)
          flag += 1

In [ ]:
n = 0
while (n < len(chistes)) & (n < len(jokes)):
  dat = {"Joke": (jokes["Jokes"][n]), "Chiste": (chistes["Chiste"][n])}
  post = post.append(dat, ignore_index=True)
  n += 1
post.to_excel("/content/drive/MyDrive/Trabajo Fin de Grado/Final-Degree-Proyect/Wrappers/Jokes.xlsx")